#### Loading and preparing the PCam data for training shallow learning models using tensorflow dataset (tfds)

In [1]:
!pip install sklearn
!pip install matplotlib
!pip install tensorflow
!pip install tensorflow_datasets

Loading the required libraries

In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
from sklearn.preprocessing import StandardScaler

Defining a function that grayscale, resize and flattens the image. This function might also become handy (for deep learning) if the original images are too large for your hardware configuration.

In [2]:
def convert_sample(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image,[32,32]).numpy()
    image = image.reshape(1,-1)
    return image

You can use your google drive to store the data by "mounting" it as follows

In [4]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


Next we use the tensorflow dataset API - tfds - to load data from your mounted google drive. Note this API requite that you should have copied the entire **patch_camelyon** folder from https://syddanskuni-my.sharepoint.com/:f:/g/personal/cmd_sam_sdu_dk/EiWD2LmuxCJBp-_tfGK7aL8Bair7l5z8FU5sp5pLjlhKwg?e=FLzWno to the /content/drive/MyDrive folder on your google drive:

In [15]:
data_path = 'C:/Users/andly/Desktop/DATA SCIENCE NOTER/AML/exam/patch_camelyon/'
print(data_path)
ds1,ds2,ds3 = tfds.load('patch_camelyon',
                        split=['train[:5%]','test[:2%]','validation[:2%]'],
                        data_dir = data_path,
                        download=False,
                        batch_size=-1, # All data...no batches needed 
                        as_supervised=True, # So that we easily can transform data to numpy format
                        shuffle_files=True)

C:/Users/andly/Desktop/DATA SCIENCE NOTER/AML/exam/patch_camelyon/


AssertionError: Dataset patch_camelyon: could not find data in C:/Users/andly/Desktop/DATA SCIENCE NOTER/AML/exam/patch_camelyon/. Please make sure to call dataset_builder.download_and_prepare(), or pass download=True to tfds.load() before trying to access the tf.data.Dataset object.

Next we can easily convert both the images and the labels to numpy format 

In [6]:
train_dataset       = tfds.as_numpy(ds1)
train_dataset_image = np.vstack(list(map(convert_sample,train_dataset[0])))
train_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(train_dataset_image)
train_dataset_label = train_dataset[1].reshape(-1,)    
print(f'Shape of training data features (observations,features): {train_dataset_image_Scaled.shape}')
print(f'Shape of training data labels (observations,): {train_dataset_label.shape}')

validation_dataset  = tfds.as_numpy(ds3)
validation_dataset_image = np.vstack(list(map(convert_sample,validation_dataset[0])))
validation_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(validation_dataset_image)
validation_dataset_label = validation_dataset[1].reshape(-1,) 
   
test_dataset        = tfds.as_numpy(ds2)
test_dataset_image = np.vstack(list(map(convert_sample,test_dataset[0])))
test_dataset_image_Scaled = StandardScaler(with_mean=0, with_std=1).fit_transform(test_dataset_image)
test_dataset_label = test_dataset[1].reshape(-1,)

Shape of training data features (observations,features): (13107, 1024)
Shape of training data labels (observations,): (13107,)


The data is then ready to be applied for training, validation, testing in a shallow learning model such as the SVM classifier...below just a very very simple illustration on how to construct and train a support vector machine based on the data we have prepared

In [7]:
clf = svm.SVC(kernel='rbf')
clf.fit(train_dataset_image_Scaled, train_dataset_label)
y_test_hat = clf.predict(test_dataset_image)

# Obtain accuracy by using the `accuracy_score` function
accuracy_linear = accuracy_score(y_test_hat, test_dataset_label )
# Print results
print(f'SVM achieved {round(accuracy_linear * 100, 1)}% accuracy.')

SVM achieved 51.9% accuracy.
